In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

match_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("season", IntegerType(), True),
    StructField("city", StringType(), True),
    StructField("date", StringType(), True),   # <-- STRING here
    StructField("team1", StringType(), True),
    StructField("team2", StringType(), True),
    StructField("toss_winner", StringType(), True),
    StructField("toss_decision", StringType(), True),
    StructField("result", StringType(), True),
    StructField("dl_applied", IntegerType(), True),
    StructField("winner", StringType(), True),
    StructField("win_by_runs", IntegerType(), True),
    StructField("win_by_wickets", IntegerType(), True),
    StructField("player_of_match", StringType(), True),
    StructField("venue", StringType(), True),
    StructField("umpire1", StringType(), True),
    StructField("umpire2", StringType(), True),
    StructField("umpire3", StringType(), True),
    StructField("home_team", StringType(), True)
])


In [ ]:

from pyspark.sql.functions import *
from pyspark.sql.types import *

df_match = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.schemaLocation", "/mnt/schema/match")
    .option("header", "true")
    .schema(match_schema)
    .load("abfss://ipl-incremental-data@incrementalipldata.dfs.core.windows.net/row/match/")
)

In [ ]:
df_match_final = (
    df_match
    .withColumnRenamed("id", "match_id")
    .withColumn("source_file", col("_metadata.file_path"))
    .withColumn("ingestion_ts", current_timestamp())
)


In [ ]:
(
    df_match_final.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", "/mnt/checkpoints/match")
    .table("bronze.match_loading")
)